The following Jupyter-Notebook shows how to train a simple PyTorch classifier on the **German Character Recognition Dataset**. The trained network achieves an ACC- and MCC-value of more than 0.99.   

First we define some global variables which are used for the whole training process

In [1]:
import torch
# Change the paths accordingly
path_train_csv = "/kaggle/input/german-character-recognition-dataset/train.csv"
path_test_csv = "/kaggle/input/german-character-recognition-dataset/test.csv"
# First we have to select the classes on which we would like to train on
classes = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
# All available classes which are contained in the dataset
#classes = ['!','$','&','(',')','+','0','1','2','3','4','5','6','7','8','9','<','>','?','A','B','C','D','E','F','G','H',
#           'I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i',
#           'j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','~','ß','α','β','π','φ','€','∑','√','∞',
#           '∫']
dict_classes_to_numbers = dict(zip(classes, range(len(classes))))
dict_numbers_to_classes = dict(zip(range(len(classes)), classes))
num_classes = len(classes)
print("Num classes: " + str(num_classes))
num_val_samples_per_class = 250
# Standard DL-parameters
batch_size_train = 128
batch_size_val = 256
num_workers = 2
lr = 0.001
hparams = {"num_epochs": 100, "early_stopping_patience": 5, "early_stopping_threshold": 0.001}
# For getting reproducible results
seed = 0
torch.manual_seed(seed)

Num classes: 10


We define some helper functions for the training

In [2]:
import torchmetrics

class EarlyStopper:
    def __init__(self, patience=5, min_delta=0.001, model_weights=None):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_validation_acc = 0
        self.best_model_weights = model_weights

    def early_stop(self, validation_acc, model_weights):
       # If the model improved we store the best weights
        if validation_acc > self.best_validation_acc:
            self.best_model_weights = model_weights
        
        # If the model improved more than the threshold  
        if validation_acc > self.best_validation_acc + self.min_delta:
            self.counter = 0
            self.best_validation_acc = validation_acc
        else:          
            if self.counter >= self.patience:
                return True
            self.counter += 1    
        return False


class EpochInformation:
    def __init__(self, model, device, num_classes, dataset_sizes):
        self.mcc_metric = torchmetrics.MatthewsCorrCoef(task='multiclass', num_classes=num_classes).to(device)
        self.auc_metric = torchmetrics.AUROC(task='multiclass', num_classes=num_classes).to(device)
        self.dataset_sizes = dataset_sizes
        self.running_loss = 0.0
        self.running_outputs = None
        self.running_labels = None
        self.model = model
        
    def reset_metrics(self):
        self.running_loss = 0.0
        self.running_outputs = None
        self.running_labels = None            
        
    def update_metrics_for_batch(self, outputs, loss, inputs, labels):
        if self.running_outputs is None:
            self.running_outputs = outputs
        else:
            self.running_outputs = torch.cat((self.running_outputs, outputs), dim=0)
        
        if self.running_labels is None:
            self.running_labels = labels
        else:
            self.running_labels = torch.cat((self.running_labels, labels), dim=0)

        #update the loss
        self.running_loss += loss.item() * inputs.size(0)
        
    def calculate_metrics(self, phase):
        loss = self.running_loss / self.dataset_sizes[phase]
                
        _, predictions = torch.max(self.running_outputs, 1)
        comparison = predictions == self.running_labels
        corrects = torch.sum(comparison)
        
        acc = corrects.double() / self.dataset_sizes[phase]
        mcc = self.mcc_metric(predictions, self.running_labels)
        auc = self.auc_metric(self.running_outputs, self.running_labels)
        
        # The gradient norm can only be calculated during training
        # Also we calculate the weight-norm only once in each training epoch
        if self.model.training:
            grads = [param.grad.detach().flatten() for param in self.model.parameters() if param.grad is not None]
            l2_norm_grads = torch.linalg.vector_norm(torch.cat(grads))
            weights = [param.detach().flatten() for param in self.model.parameters()]
            l2_norm_weights = torch.linalg.vector_norm(torch.cat(weights)) 
           
        result_dict = {
            "loss" : loss,
            "acc"  : acc.item(),
            "mcc"  : mcc.item(),
            "auc"  : auc.item()
        }
        if self.model.training:
            result_dict["l2_grad"] = l2_norm_grads.item()
            result_dict["l2_weights"] = l2_norm_weights.item()
            
        return result_dict 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Now we define a simple CNN-model

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PyTorchClassifier(nn.Module):
    def __init__(self, num_classes):
        super(PyTorchClassifier, self).__init__()
        self.size_fc1 = 1024
        self.conv1 = nn.Conv2d(1, 32, 6, padding="same")
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 4, padding="same")
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 4, padding="same")
        self.pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(128, 256, 2, padding="same")
        self.conv5 = nn.Conv2d(256, 256, 2)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(self.size_fc1, 256)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = self.pool4(F.relu(self.conv5(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.dropout1(x)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

The code for generating the dataset

In [4]:
from torch.utils.data import Dataset
import numpy as np

class GermanCharacterRecognitionDS(Dataset):
    def __init__(self, path_csv, transform=None, target_transform=None, classes=[],
                 num_channels=1):
        self.path_csv = path_csv
        self.transform = transform
        self.target_transform = target_transform
        self.data_lines = self.read_lines_csv(classes)
        self.n = len(self.data_lines)
        self.classes = classes
        self.num_channels = num_channels

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        label, image = self.parse_one_line(idx)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)            
        # We have to convert the label to an integer value
        label = dict_classes_to_numbers[label]
        return image, label

    def read_lines_csv(self, classes):
        training_data_file = open(self.path_csv, 'r')
        data_lines = training_data_file.readlines()
        training_data_file.close()
        data_lines = [line for line in data_lines if line[0] in classes]
        return data_lines

    def parse_one_line(self, index):
        line = self.data_lines[index].split(',')
        image_np = np.asarray(line[1:1601], dtype=np.float32)
        image_np = image_np.reshape(40, 40, 1)
        if self.num_channels != 1:
            image_np = np.repeat(image_np, self.num_channels, axis=2)
        return line[0], image_np

Then we can define the train loop

In [5]:
import copy
from tqdm import tqdm
from torchvision import transforms

def train_model(data_loaders, model, loss_func, optimizer, device):
    print("training started")
    num_epochs = hparams["num_epochs"]
    information = EpochInformation(model, device, num_classes, dataset_sizes)
    early_stopper = EarlyStopper(patience=hparams["early_stopping_patience"],
                             min_delta=hparams["early_stopping_threshold"],
                             model_weights=copy.deepcopy(model.state_dict()))
    strop_training = False
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        if strop_training == True:
            break
        # Each epoch has a training and validation phase
        for phase in ['val', 'train']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()  
            information.reset_metrics()
            
            if phase == 'train':
                print("training...")
            else:
                print("validating...")                
            data_loader = tqdm(data_loaders[phase])
            for inputs, labels in data_loader:
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = loss_func(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                information.update_metrics_for_batch(outputs, loss, inputs, labels)

            result_dict = information.calculate_metrics(phase)
            # prints the all metrics of the training and validation phase
            print(" ".join(name + ": " + str(round(value, 4)) for name, value in result_dict.items()))

            if phase == 'val':
                if early_stopper.early_stop(result_dict["mcc"], copy.deepcopy(model.state_dict())):
                    print('early stopping')
                    strop_training = True
    # load best model
    model.load_state_dict(early_stopper.best_model_weights) 
    return model

For loading the data we need some helper methods. As stated in the description of the dataset, the representation of each class of the train data set is the same for each class. We also want to make sure that the validation data has the same distribution as the test data, so we need a function which takes a certain amount of samples from each class of the train data set and puts them into the validation data set. To optimize the run time, we save the indices of the train and validation data set in a numpy array. This approach eliminates the need to regenerate the data split each time, thereby significantly reducing processing time.

In [6]:
from torch.utils.data import Subset

def get_train_and_val_set(train_data_set, num_samples_validation_data=250):
    # Define the ratio for train and validation data
    print("Splitting train- and val-data ...")
    val_count = dict(zip(classes, len(classes) * [0]))
    val_indices = []
    train_indices = []
    for i in range(len(train_data_set)):
        _, label = train_data_set[i]
        label_string = dict_numbers_to_classes[label]
        if val_count[label_string] < num_samples_validation_data:
            val_count[label_string] += 1
            val_indices.append(i)
        else:
            train_indices.append(i)

    np.save("val_indices.npy", np.asarray(val_indices))
    np.save("train_indices.npy", np.asarray(train_indices))
    train_set, val_set = split_train_set(train_data_set, train_indices, val_indices)
    print("Splitting done")
    return train_set, val_set

def split_train_set(train_data_set, train_indices, val_indices):
    train_set = Subset(train_data_set, train_indices)
    val_set = Subset(train_data_set, val_indices)
    return train_set, val_set

def get_class_counts_of_data_loader(data_loader):
    labels_count_dict = dict(zip(classes, len(classes) * [0]))
    for _, labels in data_loader:
        string_labels = [dict_numbers_to_classes[number] for number in labels.tolist()]
        for label in string_labels:
            labels_count_dict[label] += 1
    return labels_count_dict

Now, we can proceed to construct all the necessary data loaders.

In [7]:
from random import random

# We normalize with the men and std of the train set
standard_transforms = [transforms.ToTensor(),transforms.Normalize(35.37502147246886, 75.87412766890324)]
test_set = GermanCharacterRecognitionDS(path_test_csv, transform=transforms.Compose(standard_transforms), classes=classes,
                                        num_channels=1)
train_set = GermanCharacterRecognitionDS(path_train_csv, transform=None, classes=classes,
                                         num_channels=1)
num_train = len(train_set)
num_test = len(test_set)
print("number Train set: " + str(num_train))
print("number Test set: " + str(num_test))
print("number whole ds: " + str(num_test + num_train))
# TODO comment the following line after the first run
train_set, val_set = get_train_and_val_set(train_set, num_val_samples_per_class)
# TODO uncomment this line if you want to use the precalculated indices which speeds up the run time
# train_set, val_set = split_train_set(train_set, np.load("train_indices.npy"), np.load("val_indices.npy"))

train_transforms = standard_transforms + [transforms.RandomRotation(30), transforms.RandomGrayscale(p=0.1), 
                                          transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))]
train_set.dataset.transform = transforms.Compose(train_transforms)
val_set.dataset.transform = transforms.Compose(standard_transforms)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size_train, shuffle=True, num_workers=num_workers,                                                   generator=g)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size_val, shuffle=False, num_workers=num_workers,
                                         generator=g)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size_val, shuffle=False, num_workers=num_workers,
                                          generator=g)

class_counts_train = get_class_counts_of_data_loader(train_loader)
class_counts_val = get_class_counts_of_data_loader(val_loader)
class_counts_test = get_class_counts_of_data_loader(test_loader)

print("train_loader: " + str(class_counts_train))
print("val_loader: " + str(class_counts_val))
print("test_loader: " + str(class_counts_test))

data_loaders = {"train": train_loader, "val": val_loader, "test": test_loader}
dataset_sizes = {"train": len(train_loader.dataset), "val": len(val_loader.dataset), "test": len(test_loader.dataset)}

number Train set: 44251
number Test set: 5000
number whole ds: 49251
Splitting train- and val-data ...
Splitting done
train_loader: {'0': 4541, '1': 4203, '2': 4168, '3': 4120, '4': 4019, '5': 3966, '6': 4235, '7': 4161, '8': 4213, '9': 4125}
val_loader: {'0': 250, '1': 250, '2': 250, '3': 250, '4': 250, '5': 250, '6': 250, '7': 250, '8': 250, '9': 250}
test_loader: {'0': 500, '1': 500, '2': 500, '3': 500, '4': 500, '5': 500, '6': 500, '7': 500, '8': 500, '9': 500}


We also calculate the class weights in order to use a weighted loss function

In [8]:
class_weights = []
number_train_values = len(train_loader.dataset)
for class_label in classes:
    weight = float(number_train_values) / class_counts_train[class_label]
    class_weights.append(weight)
class_weights = torch.tensor(class_weights)
sum_class_weights = torch.sum(class_weights)
class_weights = class_weights / sum_class_weights
print("class weights: ", str(dict(zip(classes, class_weights.tolist()))))

class weights:  {'0': 0.09183375537395477, '1': 0.09921891242265701, '2': 0.10005208849906921, '3': 0.10121773928403854, '4': 0.10376140475273132, '5': 0.10514803230762482, '6': 0.09846921265125275, '7': 0.10022040456533432, '8': 0.09898340702056885, '9': 0.101095050573349}


Building the model

In [9]:
#from torchsummary import summary
model = PyTorchClassifier(len(classes))
# print the model
#summary(model, (1, 40, 40))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# Put the tensors on the GPU
model.to(device)

cuda:0


PyTorchClassifier(
  (conv1): Conv2d(1, 32, kernel_size=(6, 6), stride=(1, 1), padding=same)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(1, 1), padding=same)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=same)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1), padding=same)
  (conv5): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)

Now we can start the training

In [10]:
class_weights = class_weights.to(device)
optimizer = torch.optim.NAdam(model.parameters(), lr=lr)
loss_func = torch.nn.CrossEntropyLoss(weight=class_weights)
model = train_model(data_loaders, model, loss_func, optimizer, device)

training started
Epoch 0/99
----------
validating...


  0%|          | 0/10 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/Convolution.cpp:1003.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 10/10 [00:05<00:00,  1.75it/s]


loss: 2.3035 acc: 0.1 mcc: 0 auc: 0.5338
training...


100%|██████████| 327/327 [00:19<00:00, 17.02it/s]


loss: 0.3656 acc: 0.8802 mcc: 0.8671 auc: 0.9916 l2_grad: 1.3001 l2_weights: 23.8132
Epoch 1/99
----------
validating...


...

loss: 0.019 acc: 0.9946 mcc: 0.994 auc: 1.0 l2_grad: 6.7176 l2_weights: 55.1883
Epoch 16/99
----------
validating...


100%|██████████| 10/10 [00:01<00:00,  8.58it/s]


loss: 0.0858 acc: 0.9792 mcc: 0.9769 auc: 0.9996
early stopping
training...


100%|██████████| 327/327 [00:18<00:00, 17.90it/s]

loss: 0.0171 acc: 0.9948 mcc: 0.9943 auc: 1.0 l2_grad: 0.3753 l2_weights: 56.9699
Epoch 17/99
----------


After the training we evaluate the model

In [11]:
information_test = EpochInformation(model, device, num_classes, dataset_sizes)
model.eval()
for inputs, labels in data_loaders["test"]:
    inputs = inputs.to(device, non_blocking=True)
    labels = labels.to(device, non_blocking=True)
    optimizer.zero_grad()
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        loss = loss_func(outputs, labels)
    information_test.update_metrics_for_batch(outputs, loss, inputs, labels)

result_dict = information_test.calculate_metrics("test")
print("Test metrics:")
print(" ".join(name + ": " + str(round(value, 4)) for name, value in result_dict.items()))

Test metrics:
loss: 0.0509 acc: 0.9884 mcc: 0.9871 auc: 0.9998
